# Analysis

In [ ]:
tables = [
    "careplans",
    "conditions",
    "devices",
    "encounters",
    "imaging_studies",
    "immunizations",
    "medications",
    "observations",
    "patients",
    "procedures",
]

In [ ]:
!mkdir -p data/allergy

In [3]:
from urllib.request import urlopen
import os

files = [
    "data/allergy/careplans.csv",
    "data/allergy/conditions.csv",
    "data/allergy/devices.csv",
    "data/allergy/disease.csv",
    "data/allergy/encounters.csv",
    "data/allergy/imaging_studies.csv",
    "data/allergy/immunizations.csv",
    "data/allergy/medications.csv",
    "data/allergy/observations.csv",
    "data/allergy/patients.csv",
    "data/allergy/payers.csv",
    "data/allergy/procedures.csv",
    "data/allergy/providers.csv",
    "data/allergy/supplies.csv",
]

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        print("File {} already exists, skipping download".format(filename))
    else:
        print("Downloading {}".format(filename))

    download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

for file in files:
    ensure_file_has_been_downloaded(file)

File data/allergy/careplans.csv already exists, skipping download
File data/allergy/conditions.csv already exists, skipping download
File data/allergy/devices.csv already exists, skipping download
File data/allergy/disease.csv already exists, skipping download
File data/allergy/encounters.csv already exists, skipping download
File data/allergy/imaging_studies.csv already exists, skipping download
File data/allergy/immunizations.csv already exists, skipping download
File data/allergy/medications.csv already exists, skipping download
File data/allergy/observations.csv already exists, skipping download
File data/allergy/patients.csv already exists, skipping download
File data/allergy/payers.csv already exists, skipping download
File data/allergy/procedures.csv already exists, skipping download
File data/allergy/providers.csv already exists, skipping download
File data/allergy/supplies.csv already exists, skipping download


In [3]:
import extract

tables = [
    "careplans",
    "conditions",
    "devices",
    "disease",
    "encounters",
    "imaging_studies",
    "immunizations",
    "medications",
    "observations",
    "patients",
    "procedures",
]

connection = extract.connect_to_db("allergy.sqlite")
for table in tables:
    extract.insert_values_to_table(connection.cursor(), table, "../data/allergy/" + table + ".csv")
    connection.commit()

connection.close()

SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
SQL insert process finished
